In [1]:
from __future__ import absolute_import, division
import os
import sys
import logging
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import pickle
tf.logging.set_verbosity(0)

/home/cse/dual/cs5150278/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cse/dual/cs5150278/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/cse/dual/cs5150278/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cse/dual/cs5150278/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cse/dual/cs5150278/anaconda3/lib/python3.6/importlib/_boo

In [2]:
tf.enable_eager_execution()

In [3]:
# Set PATHs
PATH_TO_SENTEVAL = '../'
PATH_TO_DATA = './data_evaluation'
sys.path.insert(0, PATH_TO_SENTEVAL)

In [4]:
# import SentEval
import senteval

/home/cse/dual/cs5150278/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
class Embedding(tf.keras.Model):
    def __init__(self, V, d):
        super(Embedding, self).__init__()
        self.W = tfe.Variable(tf.random_uniform(minval=-1.0, maxval=1.0, shape=[V, d]))
    
    def call(self, word_indexes):
        return tf.cast(tf.nn.embedding_lookup(self.W, word_indexes), tf.float32)

In [6]:
class StaticRNN(tf.keras.Model):
    def __init__(self, h, cell):
        super(StaticRNN, self).__init__()
        if cell == 'lstm':
            self.cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=h)
        elif cell == 'gru':
            self.cell = tf.nn.rnn_cell.GRUCell(num_units=h)
        else:
            self.cell = tf.nn.rnn_cell.BasicRNNCell(num_units=h)
        
        
    def call(self, state, word_vectors, num_words):
        word_vectors_time = tf.unstack(word_vectors, axis=1)
        outputs, final_state = tf.nn.static_rnn(cell=self.cell, initial_state = state, inputs=word_vectors_time, sequence_length=num_words, dtype=tf.float32)
        return outputs, final_state

In [7]:
class Encoder(tf.keras.Model):
    def __init__(self, V, d, h, cell):
        super(Encoder, self).__init__()
        self.word_embedding = Embedding(V, d)
        self.rnn = StaticRNN(h, cell)
        
    def call(self, word_vector, word_length):
        word_vectors = self.word_embedding(word_vector)
        rnn_outputs_time, final_state = self.rnn(None, word_vectors, word_length)
        output = []
        for i in range(int(tf.size(word_length))):
            output.append(rnn_outputs_time[int(word_length[i]) - 1][i])
        t = tf.convert_to_tensor(output, dtype=tf.float32)
        return t, final_state, self.word_embedding

In [8]:
def create_dataset(sentences, vocab_table, batch_size):
    sentences = tf.convert_to_tensor(sentences)
    dataset = tf.data.TextLineDataset.from_tensor_slices(sentences)
    dataset = dataset.map(lambda sentence: (
        vocab_table.lookup(tf.string_split([(tf.string_split([sentence],',')).values[0]]).values),
        tf.size(vocab_table.lookup(tf.string_split([(tf.string_split([sentence],',')).values[0]]).values))
                         ))
    dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=([None], []))
    return dataset

In [9]:
def prepare(params, samples):
    return

In [20]:
#mean of the individual word embeddings of each words has been taken
import numpy as np
def batcher(params, batch):
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []

    for sent in batch:
        sentvec = []
        for word in sent:
            if word in params['word2vec']:
                sentvec.append(params['word2vec'][word])
        if not sentvec:
            vec = np.zeros(params['wvec_dim'])
            sentvec.append(vec)
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)

    embeddings = np.vstack(embeddings)
    return embeddings

In [21]:
# define senteval params
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                 'tenacity': 3, 'epoch_size': 2}
# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

In [22]:
if __name__ == "__main__":
    
    #load the english vocab
    from tensorflow.python.ops import lookup_ops
    english_vocab_file = './data/english_vocab.txt'
    english_vocab_table = lookup_ops.index_table_from_file(english_vocab_file, default_value=0)
    keys = english_vocab_table.export()[0].numpy()
    values = english_vocab_table.export()[1].numpy()
    word2index = {k: v for k, v in zip(keys, values)}
    params_senteval['word2vec'] = word2index
    
    #assign dimensions
    params_senteval['wvec_dim'] = 256
    
    #loading the final trained model
    opt = tf.train.AdamOptimizer(learning_rate=0.002)
    encoder_nmt = Encoder(english_vocab_table.size(), 256, 512, 'gru')
    checkpoint_dir = './encoder_nmt'
    root = tfe.Checkpoint(optimizer=opt, model=encoder_nmt, optimizer_step=tf.train.get_or_create_global_step())
    root.restore(tf.train.latest_checkpoint(checkpoint_dir))
    params_senteval['word_embedding'] = encoder_nmt.word_embedding
    
    
    #running the evalutaton tasks
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks = ['MR', 'CR', 'SUBJ','MPQA', 'SST2', 'SST5', 'MRPC',
                      'SICKEntailment', 'SICKRelatedness', 'STSBenchmark',
                     'STS12', 'STS13', 'STS14', 'STS15', 'STS16']
    results = se.eval(transfer_tasks)
    print(results)

2018-10-18 02:58:50,411 : ***** Transfer task : MR *****


2018-10-18 02:58:50,415 : Generating sentence embeddings
2018-10-18 02:58:50,418 : Generated sentence embeddings
2018-10-18 02:58:50,422 : Training pytorch-MLP-nhid0-rmsprop-bs128 with (inner) 5-fold cross-validation
2018-10-18 02:58:54,733 : Best param found at split 1: l2reg = 1e-05                 with score 58.67
2018-10-18 02:58:55,004 : Best param found at split 2: l2reg = 1e-05                 with score 57.58
2018-10-18 02:58:55,275 : Best param found at split 3: l2reg = 1e-05                 with score 57.58
2018-10-18 02:58:55,546 : Best param found at split 4: l2reg = 1e-05                 with score 58.33
2018-10-18 02:58:55,816 : Best param found at split 5: l2reg = 1e-05                 with score 58.33
2018-10-18 02:58:55,832 : Dev acc : 58.1 Test acc : 58.11

2018-10-18 02:58:55,835 : ***** Transfer task : CR *****


2018-10-18 02:58:55,901 : Generating sentence embeddings
2018-10-18 02:58:56,033 : Generated sen

2018-10-18 03:01:43,728 : Test : Pearson 0 Spearman 0 MSE 1.0178068645228793                        for SICK Relatedness

2018-10-18 03:01:43,734 : 

***** Transfer task : STSBenchmark*****


2018-10-18 03:01:43,880 : Computing embedding for train
2018-10-18 03:01:44,237 : Computed train embeddings
2018-10-18 03:01:44,240 : Computing embedding for dev
2018-10-18 03:01:44,333 : Computed dev embeddings
2018-10-18 03:01:44,335 : Computing embedding for test
2018-10-18 03:01:44,421 : Computed test embeddings
2018-10-18 03:02:09,092 : Dev : Pearson 0
2018-10-18 03:02:09,095 : Test : Pearson 0 Spearman 0 MSE 2.4952048630823445                        for SICK Relatedness

2018-10-18 03:02:09,101 : ***** Transfer task : STS12 *****


/home/cse/dual/cs5150278/ankur-deep-learning/submission/senteval/utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
2018-10-18 03:02:09,251 : MSRpar : pearson = nan, spearman = n

{'MR': {'devacc': 58.1, 'acc': 58.11, 'ndev': 74, 'ntest': 74}, 'CR': {'devacc': 63.76, 'acc': 63.76, 'ndev': 3775, 'ntest': 3775}, 'SUBJ': {'devacc': 99.6, 'acc': 99.6, 'ndev': 5020, 'ntest': 5020}, 'MPQA': {'devacc': 68.77, 'acc': 68.77, 'ndev': 10606, 'ntest': 10606}, 'SST2': {'devacc': 50.92, 'acc': 49.92, 'ndev': 872, 'ntest': 1821}, 'SST5': {'devacc': 26.25, 'acc': 28.64, 'ndev': 1101, 'ntest': 2210}, 'MRPC': {'devacc': 67.54, 'acc': 66.49, 'f1': 79.87, 'ndev': 4076, 'ntest': 1725}, 'SICKEntailment': {'devacc': 56.4, 'acc': 56.69, 'ndev': 500, 'ntest': 4927}, 'SICKRelatedness': {'devpearson': 0, 'pearson': 0, 'spearman': 0, 'mse': 1.0178068645228793, 'yhat': array([3.51593995, 3.51593995, 3.51593995, ..., 3.51593995, 3.51593995,
       3.51593995]), 'ndev': 500, 'ntest': 4927}, 'STSBenchmark': {'devpearson': 0, 'pearson': 0, 'spearman': 0, 'mse': 2.4952048630823445, 'yhat': array([3.01985016, 3.01985016, 3.01985016, ..., 3.01985016, 3.01985016,
       3.01985016]), 'ndev': 1500, 